## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Castro,BR,-24.7911,-50.0119,61.38,97,14,8.34,few clouds
1,1,Great Falls,US,47.5002,-111.3008,46.67,37,100,16.11,overcast clouds
2,2,Kiama,AU,-34.6833,150.8667,85.57,54,0,1.01,clear sky
3,3,Katsuura,JP,35.1333,140.3000,46.18,61,100,16.42,overcast clouds
4,4,Egvekinot,RU,66.3167,-179.1667,-12.08,73,0,15.41,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 1
What is your desired maximum temperature for your trip? 50


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Great Falls,US,47.5002,-111.3008,46.67,37,100,16.11,overcast clouds
3,3,Katsuura,JP,35.1333,140.3000,46.18,61,100,16.42,overcast clouds
5,5,Brigantine,US,39.4101,-74.3646,37.45,70,0,8.05,clear sky
6,6,Nagato,JP,34.3833,131.2000,45.19,57,56,8.10,broken clouds
8,8,Caohai,CN,26.8661,104.2841,30.33,83,97,3.02,overcast clouds
9,9,Izhma,RU,65.0083,53.9113,8.20,92,92,10.58,overcast clouds
21,21,Kodiak,US,57.7900,-152.4072,24.31,54,0,14.97,clear sky
24,24,Roma,US,43.2128,-75.4557,28.96,74,100,10.36,overcast clouds
26,26,Ushuaia,AR,-54.8000,-68.3000,49.66,57,20,11.50,few clouds
27,27,Ketchikan,US,55.3422,-131.6461,43.27,93,100,18.41,mist


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Great Falls,US,47.5002,-111.3008,46.67,37,100,16.11,overcast clouds
3,3,Katsuura,JP,35.1333,140.3000,46.18,61,100,16.42,overcast clouds
5,5,Brigantine,US,39.4101,-74.3646,37.45,70,0,8.05,clear sky
6,6,Nagato,JP,34.3833,131.2000,45.19,57,56,8.10,broken clouds
8,8,Caohai,CN,26.8661,104.2841,30.33,83,97,3.02,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
680,680,Atbasar,KZ,51.8000,68.3333,6.85,93,100,8.19,overcast clouds
681,681,Olafsvik,IS,64.8945,-23.7142,30.65,74,100,38.43,overcast clouds
684,684,Atikokan,CA,48.7555,-91.6216,32.79,98,100,7.96,light snow
687,687,Karpogory,RU,64.0019,44.4451,17.69,93,98,8.34,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Great Falls,US,46.67,overcast clouds,47.5002,-111.3008,
3,Katsuura,JP,46.18,overcast clouds,35.1333,140.3000,
5,Brigantine,US,37.45,clear sky,39.4101,-74.3646,
6,Nagato,JP,45.19,broken clouds,34.3833,131.2000,
8,Caohai,CN,30.33,overcast clouds,26.8661,104.2841,
9,Izhma,RU,8.20,overcast clouds,65.0083,53.9113,
21,Kodiak,US,24.31,clear sky,57.7900,-152.4072,
24,Roma,US,28.96,overcast clouds,43.2128,-75.4557,
26,Ushuaia,AR,49.66,few clouds,-54.8000,-68.3000,
27,Ketchikan,US,43.27,mist,55.3422,-131.6461,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")
        

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Great Falls,US,46.67,overcast clouds,47.5002,-111.3008,La Quinta Inn & Suites by Wyndham Great Falls
3,Katsuura,JP,46.18,overcast clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
5,Brigantine,US,37.45,clear sky,39.4101,-74.3646,Legacy Vacation Resorts - Brigantine Beach
6,Nagato,JP,45.19,broken clouds,34.3833,131.2000,いそう庵
8,Caohai,CN,30.33,overcast clouds,26.8661,104.2841,Jinhua Hotel


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))